In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import os
import torch
from torchvision import transforms
from transformers import ViTFeatureExtractor, ViTForImageClassification
from PIL import Image
from torch.utils.data import DataLoader, Dataset

# Directory paths
train_path = "/kaggle/input/the-wildfire-dataset/the_wildfire_dataset/the_wildfire_dataset/train"
val_path = "/kaggle/input/the-wildfire-dataset/the_wildfire_dataset/the_wildfire_dataset/val"
test_path = "/kaggle/input/the-wildfire-dataset/the_wildfire_dataset/the_wildfire_dataset/test"

# ViT Feature Extractor
feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224-in21k')

# Custom dataset class
class WildfireDataset(Dataset):
    def __init__(self, folder_path, feature_extractor):
        self.folder_path = folder_path
        self.image_files = []
        self.labels = []
        label_mapping = {
            "nofire": 0,
            "fire": 1,
        }

        # Populating image_files and labels based on the clarified folder structure
        for class_label in ["nofire", "fire"]:
            current_path = f"{folder_path}/{class_label}"
            if class_label == "nofire":
                for subclass_label in ["Forested_areas_without_confounding_elements",
                                       "Fire_confounding_elements",
                                       "Smoke_confounding_elements"]:
                    current_subclass_path = f"{current_path}/{subclass_label}"
                    image_files = os.listdir(current_subclass_path)
                    self.image_files.extend([f"{current_subclass_path}/{img}" for img in image_files])
                    self.labels.extend([label_mapping[class_label]] * len(image_files))
            else:
                for subclass_label in ["Smoke_from_fires", "Both_smoke_and_fire"]:
                    current_subclass_path = f"{current_path}/{subclass_label}"
                    image_files = os.listdir(current_subclass_path)
                    self.image_files.extend([f"{current_subclass_path}/{img}" for img in image_files])
                    self.labels.extend([label_mapping[class_label]] * len(image_files))

        self.feature_extractor = feature_extractor

    def __getitem__(self, idx):
        img_path = self.image_files[idx]
        label = self.labels[idx]
        image = Image.open(img_path).convert("RGB")
        
        # Ensuring that the image has the correct shape (num_channels, height, width)
        image = self.feature_extractor(images=image, return_tensors="pt")["pixel_values"].squeeze(0)

        return {
            "pixel_values": image,
            "labels": torch.tensor(label),
        }

    
    def __len__(self):
        return len(self.image_files)


train_dataset = WildfireDataset(train_path, feature_extractor)
val_dataset = WildfireDataset(val_path, feature_extractor)
test_dataset = WildfireDataset(test_path, feature_extractor)


train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

2024-02-10 08:42:31.917376: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-10 08:42:31.917469: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-10 08:42:32.037335: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [2]:
import torch
from transformers import ViTForImageClassification
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
import torch.nn as nn

# Define the loss function (criterion)
criterion = nn.CrossEntropyLoss()

# Load the fine-tuned model
model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224-in21k', num_labels=2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    model = torch.nn.DataParallel(model)
model.to(device)

# Define optimizer and loss function
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

# Define training parameters
num_epochs = 5

# Store losses and accuracies
train_losses = []
accuracies = []

# Training loop
for epoch in range(num_epochs):
    model.train()
    epoch_labels = []
    epoch_predictions = []

    for batch in train_dataloader:
        inputs = batch["pixel_values"].to(device)
        labels = batch["labels"].to(device)

        optimizer.zero_grad()
        outputs = model(inputs, labels=labels)
        logits = outputs.logits
        loss = criterion(logits, labels)  # Compute the appropriate loss function here
        loss.backward()
        optimizer.step()

        train_losses.append(loss.item())


        # Store predictions and labels for accuracy calculation
        epoch_labels.extend(labels.cpu().numpy())
        epoch_predictions.extend(torch.argmax(outputs.logits, axis=1).cpu().numpy())

    # Calculate accuracy
    accuracy = accuracy_score(epoch_labels, epoch_predictions)
    accuracies.append(accuracy)

    # Print average training loss and accuracy for the epoch
    avg_train_loss = sum(train_losses[-len(train_dataloader):]) / len(train_dataloader)
    print(f"Epoch {epoch + 1}/{num_epochs}, Avg Train Loss: {avg_train_loss:.4f}, Accuracy: {accuracy:.4f}")

# Store accuracies for further analysis
print("Accuracies:", accuracies)

config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Let's use 2 GPUs!


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/PIL/Image.py:3176: DecompressionBombWarning: Image size (89747104 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/PIL/Image.py:3176: DecompressionBombWarning: Image size (104688771 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


...

Epoch 3/5, Avg Train Loss: 0.1237, Accuracy: 0.9762


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/PIL/Image.py:3176: DecompressionBombWarning: Image size (104688771 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/PIL/Image.py:3176: DecompressionBombWarning: Image size (89747104 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


Epoch 4/5, Avg Train Loss: 0.0742, Accuracy: 0.9894


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/PIL/Image.py:3176: DecompressionBombWarning: Image size (89747104 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/PIL/Image.py:3176: DecompressionBombWarning: Image size (104688771 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


Epoch 5/5, Avg Train Loss: 0.0516, Accuracy: 0.9926
Accuracies: [0.8203497615262321, 0.9470058293587705, 0.9761526232114467, 0.9894011658717541, 0.9925808161102279]


In [8]:
import torch
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
import numpy as np

# Initialize val_accuracies list
val_accuracies = []

# Validation loop
model.eval()
val_losses = []
val_preds = []
val_labels = []

with torch.no_grad():
    for batch in test_dataloader:
        inputs = batch["pixel_values"].to(device)
        labels = batch["labels"].to(device)

        # Check for non-empty batches
        if inputs.size(0) == 0:
            continue

        outputs = model(inputs, labels=labels)
        
        # Check for NaN values in the loss
        if np.any(np.isnan(outputs.loss.cpu().numpy())):
            print("NaN loss encountered. Skipping batch.")
            continue

        # Compute the loss using the appropriate criterion
        loss = criterion(outputs.logits, labels)
        val_losses.append(loss.item())

        predictions = torch.argmax(outputs.logits, dim=1)
        val_preds.extend(predictions.cpu().numpy())
        val_labels.extend(labels.cpu().numpy())

# Calculate validation accuracy
val_accuracy = accuracy_score(val_labels, val_preds)
val_loss = sum(val_losses) / len(val_losses)

print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")
print(f"Validation Loss: {val_loss:.4f}")

# Store validation accuracy for further analysis
val_accuracies.append(val_accuracy)

/opt/conda/lib/python3.10/site-packages/PIL/Image.py:3176: DecompressionBombWarning: Image size (101859328 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/PIL/Image.py:3176: DecompressionBombWarning: Image size (96631920 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/PIL/Image.py:3176: DecompressionBombWarning: Image size (94487082 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


Validation Accuracy: 97.80%
Validation Loss: 0.0730


In [10]:
# Save the fine-tuned model
if isinstance(model, torch.nn.DataParallel):
    model.module.save_pretrained("Binary_VIT")
else:
    model.save_pretrained("Binary_VIT")


In [14]:
# Create a zip archive of a folder
!zip -r binary_VIT.zip Binary_VIT

  adding: Binary_VIT/ (stored 0%)
  adding: Binary_VIT/model.safetensors (deflated 7%)
  adding: Binary_VIT/config.json (deflated 46%)


In [1]:
import os
import torch
from torchvision import transforms
from transformers import ViTFeatureExtractor, ViTForImageClassification
from PIL import Image
from torch.utils.data import DataLoader, Dataset

# Directory paths
test_path = "/kaggle/input/test-dataset/test - Copy"

# ViT Feature Extractor
feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224-in21k')

# Custom dataset class
class WildfireDataset(Dataset):
    def __init__(self, folder_path, feature_extractor):
        self.folder_path = folder_path
        self.image_files = []  # List to store image file paths
        self.labels = []  # List to store corresponding labels
        label_mapping = {
            "nofire": 0,
            "fire": 1,
        }

        # Populate image_files and labels based on the clarified folder structure
        for class_label in ["nofire", "fire"]:
            current_path = f"{folder_path}/{class_label}"
            if class_label == "nofire":
                for subclass_label in ["Forested_areas_without_confounding_elements",
                                       "Fire_confounding_elements",
                                       "Smoke_confounding_elements"]:
                    current_subclass_path = f"{current_path}/{subclass_label}"
                    image_files = os.listdir(current_subclass_path)
                    self.image_files.extend([f"{current_subclass_path}/{img}" for img in image_files])
                    self.labels.extend([label_mapping[class_label]] * len(image_files))
            else:
                for subclass_label in ["Smoke_from_fires", "Both_smoke_and_fire"]:
                    current_subclass_path = f"{current_path}/{subclass_label}"
                    image_files = os.listdir(current_subclass_path)
                    self.image_files.extend([f"{current_subclass_path}/{img}" for img in image_files])
                    self.labels.extend([label_mapping[class_label]] * len(image_files))

        self.feature_extractor = feature_extractor

    def __getitem__(self, idx):
        img_path = self.image_files[idx]
        label = self.labels[idx]
        image = Image.open(img_path).convert("RGB")
        
        # Ensure that the image has the correct shape (num_channels, height, width)
        image = self.feature_extractor(images=image, return_tensors="pt")["pixel_values"].squeeze(0)

        return {
            "pixel_values": image,
            "labels": torch.tensor(label),
        }

    
    def __len__(self):
        return len(self.image_files)

# Create dataset instances
test_dataset = WildfireDataset(test_path, feature_extractor)

# Create DataLoader instances
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

2024-03-21 08:42:54.906507: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-21 08:42:54.906618: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-21 08:42:55.036581: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [2]:
import torch
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score, matthews_corrcoef
import torch
from transformers import ViTForImageClassification
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
import torch.nn as nn

# Define the loss function (criterion)
criterion = nn.CrossEntropyLoss()

# Load the fine-tuned model
model = ViTForImageClassification.from_pretrained('/kaggle/input/capstone-models/binary_VIT/Binary_VIT', num_labels=2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    model = torch.nn.DataParallel(model)
model.to(device)

# Initialize val_accuracies list and other metrics lists
val_accuracies = []
precisions = []
recalls = []
f1_scores = []
specificities = []
fnrs = []
mccs = []
roc_auc_scores = []
conf_matrices = []

# Validation loop
model.eval()
val_losses = []
val_preds = []
val_labels = []

with torch.no_grad():
    for batch in test_dataloader:
        inputs = batch["pixel_values"].to(device)
        labels = batch["labels"].to(device)

        # Check for non-empty batches
        if inputs.size(0) == 0:
            continue

        outputs = model(inputs, labels=labels)
        
        # Check for NaN values in the loss
        if torch.isnan(outputs.loss).any():
            print("NaN loss encountered. Skipping batch.")
            continue

        # Compute the loss using the appropriate criterion
        loss = criterion(outputs.logits, labels)
        val_losses.append(loss.item())

        predictions = torch.argmax(outputs.logits, dim=1)
        val_preds.extend(predictions.cpu().numpy())
        val_labels.extend(labels.cpu().numpy())

# Calculate validation accuracy
val_accuracy = accuracy_score(val_labels, val_preds)
val_loss = sum(val_losses) / len(val_losses)

# Calculate precision, recall, F1-score, specificity, FNR, MCC, ROC-AUC, and confusion matrix
precision = precision_score(val_labels, val_preds)
recall = recall_score(val_labels, val_preds)
f1 = f1_score(val_labels, val_preds)
conf_matrix = confusion_matrix(val_labels, val_preds)
tn, fp, fn, tp = conf_matrix.ravel()
specificity = tn / (tn + fp)
fnr = fn / (fn + tp)
mcc = matthews_corrcoef(val_labels, val_preds)
roc_auc = roc_auc_score(val_labels, val_preds)

# Print metrics
print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")
print(f"Specificity: {specificity:.4f}")
print(f"FNR: {fnr:.4f}")
print(f"MCC: {mcc:.4f}")
print(f"ROC-AUC: {roc_auc:.4f}")
print(f"Confusion Matrix:\n{conf_matrix}")

# Store metrics for further analysis
val_accuracies.append(val_accuracy)
precisions.append(precision)
recalls.append(recall)
f1_scores.append(f1)
specificities.append(specificity)
fnrs.append(fnr)
mccs.append(mcc)
roc_auc_scores.append(roc_auc)
conf_matrices.append(conf_matrix)


Let's use 2 GPUs!


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/PIL/Image.py:3176: DecompressionBombWarning: Image size (101859328 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/PIL/Image.py:3176: DecompressionBombWarning: Image size (96631920 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/PIL/Image.py:3176: DecompressionBombWarning: Image size (94487082 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


Validation Accuracy: 97.80%
Precision: 0.9747
Recall: 0.9686
F1-score: 0.9716
Specificity: 0.9841
FNR: 0.0314
MCC: 0.9537
ROC-AUC: 0.9763
Confusion Matrix:
[[247   4]
 [  5 154]]


In [3]:
import os
import torch
from torchvision import transforms
from transformers import ViTFeatureExtractor, ViTForImageClassification
from PIL import Image
from torch.utils.data import DataLoader, Dataset

# Directory paths
train_path = "/kaggle/input/train-original/train - Copy"

# ViT Feature Extractor
feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224-in21k')

# Custom dataset class
class WildfireDataset(Dataset):
    def __init__(self, folder_path, feature_extractor):
        self.folder_path = folder_path
        self.image_files = []  # List to store image file paths
        self.labels = []  # List to store corresponding labels
        label_mapping = {
            "nofire": 0,
            "fire": 1,
        }

        # Populate image_files and labels based on the clarified folder structure
        for class_label in ["nofire", "fire"]:
            current_path = f"{folder_path}/{class_label}"
            if class_label == "nofire":
                for subclass_label in ["Forested_areas_without_confounding_elements",
                                       "Fire_confounding_elements",
                                       "Smoke_confounding_elements"]:
                    current_subclass_path = f"{current_path}/{subclass_label}"
                    image_files = os.listdir(current_subclass_path)
                    self.image_files.extend([f"{current_subclass_path}/{img}" for img in image_files])
                    self.labels.extend([label_mapping[class_label]] * len(image_files))
            else:
                for subclass_label in ["Smoke_from_fires", "Both_smoke_and_fire"]:
                    current_subclass_path = f"{current_path}/{subclass_label}"
                    image_files = os.listdir(current_subclass_path)
                    self.image_files.extend([f"{current_subclass_path}/{img}" for img in image_files])
                    self.labels.extend([label_mapping[class_label]] * len(image_files))

        self.feature_extractor = feature_extractor

    def __getitem__(self, idx):
        img_path = self.image_files[idx]
        label = self.labels[idx]
        image = Image.open(img_path).convert("RGB")
        
        # Ensure that the image has the correct shape (num_channels, height, width)
        image = self.feature_extractor(images=image, return_tensors="pt")["pixel_values"].squeeze(0)

        return {
            "pixel_values": image,
            "labels": torch.tensor(label),
        }

    
    def __len__(self):
        return len(self.image_files)

# Create dataset instances
train_dataset = WildfireDataset(train_path, feature_extractor)

# Create DataLoader instances
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=False)

/opt/conda/lib/python3.10/site-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [4]:
import torch
from transformers import ViTForImageClassification
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score

# Load the fine-tuned model
model = ViTForImageClassification.from_pretrained('/kaggle/input/capstone-models/binary_VIT/Binary_VIT', num_labels=2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    model = torch.nn.DataParallel(model)
model.to(device)

# Define optimizer and loss function
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
loss_fn = torch.nn.CrossEntropyLoss()  # Cross-entropy loss handles labels starting from 0

# Define training parameters
num_epochs = 5

# for param in model.base_model.parameters():
#     param.requires_grad = False  # Freeze initial layers


# Training loop
start_epoch = 6  # Start from the next epoch after the previous training

for epoch in range(start_epoch, start_epoch + num_epochs):
    model.train()
    train_losses = []

    for batch in train_dataloader:
        inputs = batch["pixel_values"].to(device)
        labels = batch["labels"].to(device)

        optimizer.zero_grad()
        outputs = model(inputs, labels=labels)
        logits = outputs.logits
        loss = loss_fn(logits, labels)
        loss.backward()
        optimizer.step()

        train_losses.append(loss.item())

    # Print average training loss for the epoch4
    avg_train_loss = sum(train_losses) / len(train_losses)
    print(f"Epoch {epoch + 1}/{num_epochs}, Avg Train Loss: {avg_train_loss:.4f}")

Let's use 2 GPUs!


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/PIL/Image.py:3176: DecompressionBombWarning: Image size (104688771 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/PIL/Image.py:3176: DecompressionBombWarning: Image size (89747104 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


Epoch 7/5, Avg Train Loss: 0.4021
Epoch 8/5, Avg Train Loss: 0.2940
Epoch 9/5, Avg Train Loss: 0.1285
Epoch 10/5, Avg Train Loss: 0.0841
Epoch 11/5, Avg Train Loss: 0.0530


In [5]:
import torch
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score, matthews_corrcoef
import torch
from transformers import ViTForImageClassification
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
import torch.nn as nn

# Define the loss function (criterion)
criterion = nn.CrossEntropyLoss()

# model = ViTForImageClassification.from_pretrained('/kaggle/input/capstone-models/binary_VIT/Binary_VIT', num_labels=2)
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# if torch.cuda.device_count() > 1:
#     print("Let's use", torch.cuda.device_count(), "GPUs!")
#     model = torch.nn.DataParallel(model)
# model.to(device)

# Initialize val_accuracies list and other metrics lists
val_accuracies = []
precisions = []
recalls = []
f1_scores = []
specificities = []
fnrs = []
mccs = []
roc_auc_scores = []
conf_matrices = []

# Validation loop
model.eval()
val_losses = []
val_preds = []
val_labels = []

with torch.no_grad():
    for batch in test_dataloader:
        inputs = batch["pixel_values"].to(device)
        labels = batch["labels"].to(device)

        # Check for non-empty batches
        if inputs.size(0) == 0:
            continue

        outputs = model(inputs, labels=labels)
        
        # Check for NaN values in the loss
        if torch.isnan(outputs.loss).any():
            print("NaN loss encountered. Skipping batch.")
            continue

        # Compute the loss using the appropriate criterion
        loss = criterion(outputs.logits, labels)
        val_losses.append(loss.item())

        predictions = torch.argmax(outputs.logits, dim=1)
        val_preds.extend(predictions.cpu().numpy())
        val_labels.extend(labels.cpu().numpy())

# Calculate validation accuracy
val_accuracy = accuracy_score(val_labels, val_preds)
val_loss = sum(val_losses) / len(val_losses)

# Calculate precision, recall, F1-score, specificity, FNR, MCC, ROC-AUC, and confusion matrix
precision = precision_score(val_labels, val_preds)
recall = recall_score(val_labels, val_preds)
f1 = f1_score(val_labels, val_preds)
conf_matrix = confusion_matrix(val_labels, val_preds)
tn, fp, fn, tp = conf_matrix.ravel()
specificity = tn / (tn + fp)
fnr = fn / (fn + tp)
mcc = matthews_corrcoef(val_labels, val_preds)
roc_auc = roc_auc_score(val_labels, val_preds)

# Print metrics
print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")
print(f"Specificity: {specificity:.4f}")
print(f"FNR: {fnr:.4f}")
print(f"MCC: {mcc:.4f}")
print(f"ROC-AUC: {roc_auc:.4f}")
print(f"Confusion Matrix:\n{conf_matrix}")

# Store metrics for further analysis
val_accuracies.append(val_accuracy)
precisions.append(precision)
recalls.append(recall)
f1_scores.append(f1)
specificities.append(specificity)
fnrs.append(fnr)
mccs.append(mcc)
roc_auc_scores.append(roc_auc)
conf_matrices.append(conf_matrix)


/opt/conda/lib/python3.10/site-packages/PIL/Image.py:3176: DecompressionBombWarning: Image size (101859328 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/PIL/Image.py:3176: DecompressionBombWarning: Image size (96631920 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/PIL/Image.py:3176: DecompressionBombWarning: Image size (94487082 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


Validation Accuracy: 97.80%
Precision: 0.9808
Recall: 0.9623
F1-score: 0.9714
Specificity: 0.9880
FNR: 0.0377
MCC: 0.9537
ROC-AUC: 0.9752
Confusion Matrix:
[[248   3]
 [  6 153]]


In [6]:
# Save the fine-tuned model
if isinstance(model, torch.nn.DataParallel):
    model.module.save_pretrained("Binary_VIT_10epochs")
else:
    model.save_pretrained("Binary_VIT_10epochs")


In [7]:
# Create a zip archive of a folder
!zip -r Binary_VIT_10epochs.zip Binary_VIT_10epochs

  adding: Binary_VIT_10epochs/ (stored 0%)
  adding: Binary_VIT_10epochs/model.safetensors (deflated 7%)
  adding: Binary_VIT_10epochs/config.json (deflated 46%)
